```
print("Size of adjacency matrix = " + str(adj.shape))
print("Size of feature matrix = " + str(features.shape))     # expected to be sparse
print("Size of label matrix = " + str(train_mask.shape))
print(y_train.shape)
print(y_val.shape)
print(y_test.shape)
print(train_mask.shape)
print(val_mask.shape)
```
produces these results:
```
Size of adjacency matrix = (2708, 2708)
Size of feature matrix = (2708, 1433)
Size of label matrix = (2708,)
(2708, 7)
(2708, 7)
(2708, 7)
(2708,)
(2708,)
```

A mask is used which is why they are all the same size

In [16]:
#### Short demonstation of how Hamming loss will be calculated
import tensorflow as tf

thresh = tf.constant(0.5)                                   # define a threshold
sample_output = tf.constant([0.32,0.68,0.43,0.5,0.98])      # sample logit output
preds = tf.math.greater(sample_output, thresh, name=None)   # prediciton based on output and threshold
print(preds)

labels = tf.constant([0, 1, 1, 0, 1])                       # define samplel labels
bool_labels = tf.equal(labels,tf.constant(1))               # transform to bool
print(bool_labels)
matching = tf.math.logical_and(bool_labels, preds)          # logical and used to get intersection
print(matching)
intersect_size = tf.reduce_sum(tf.cast(matching, tf.float32)) # count how many true in intersection of label
print(intersect_size)
num_labels = tf.size(labels)                                    # find numbe rof labels

ham_loss = tf.divide(intersect_size, tf.cast(num_labels, tf.float32))
print(ham_loss)

tf.Tensor([False  True False False  True], shape=(5,), dtype=bool)
tf.Tensor([False  True  True False  True], shape=(5,), dtype=bool)
tf.Tensor([False  True False False  True], shape=(5,), dtype=bool)
tf.Tensor(2.0, shape=(), dtype=float32)
tf.Tensor(0.4, shape=(), dtype=float32)


In [26]:
# Short demonstation of how Hamming loss will be calculated for multiple vectors
import tensorflow as tf

# define a threshold
thresh = tf.constant(0.5)
sample_output = tf.constant([[0.23, 0.98, 0.78, 0.52, 0.48], [0.1, 0.2, 0.3, 0.6, 0.9], [
                     0.9, 0.1, 0.99, 0.0, 0.03]])      # sample logit output
# prediciton based on output and threshold
preds = tf.math.greater(sample_output, thresh, name=None)
print(preds)

# define samplel labels
labels = tf.constant([[0, 1, 1, 0, 1], [0, 0, 0, 0, 1], [1, 0, 1, 0, 0]])
bool_labels = tf.equal(labels, tf.constant(
    1))               # transform to bool
print(bool_labels)
# logical and used to get intersection
matching = tf.math.logical_and(bool_labels, preds)
print(matching)
# count how many true in intersection of label
intersect_size = tf.reduce_sum(tf.cast(matching, tf.float32),axis=1)
print(intersect_size)
# find numbe rof labels
num_labels = tf.constant(labels.get_shape()[1])
print(num_labels)
ham_loss_all = tf.divide(intersect_size, tf.cast(num_labels, tf.float32))
print(ham_loss_all)
print(tf.size(labels))


tf.Tensor(
[[False  True  True  True False]
 [False False False  True  True]
 [ True False  True False False]], shape=(3, 5), dtype=bool)
tf.Tensor(
[[False  True  True False  True]
 [False False False False  True]
 [ True False  True False False]], shape=(3, 5), dtype=bool)
tf.Tensor(
[[False  True  True False False]
 [False False False False  True]
 [ True False  True False False]], shape=(3, 5), dtype=bool)
tf.Tensor([2. 1. 2.], shape=(3,), dtype=float32)
tf.Tensor(5, shape=(), dtype=int32)
tf.Tensor([0.4 0.2 0.4], shape=(3,), dtype=float32)
tf.Tensor(15, shape=(), dtype=int32)


5